<a href="https://colab.research.google.com/github/KwonHo-geun/AI_Study/blob/main/Face_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 얼굴 데이터셋과 랜드마크(landmark)

## 1. 얼굴 데이터셋을 다루는 이유
- 얼굴 인식 및 분석은 **보안, 감정 분석, HCI(Human-Computer Interaction), AR/VR** 등 다양한 응용 분야에서 활용됨.
- 데이터셋은 모델이 얼굴 구조와 패턴을 학습하는 데 핵심적 역할을 함.
- 예시:
  - 얼굴 검출 (Face Detection)
  - 얼굴 정렬 (Face Alignment)
  - 표정/감정 분석
  - 신원 인식 (Face Recognition)

## 2. Landmark란?
- **Landmark**는 얼굴의 중요한 특징점(keypoint)을 의미.
- 일반적으로 눈, 코, 입, 턱선 등 위치를 점으로 표시.
- Landmark는 얼굴 정렬(얼굴 각도 보정), 표정 분석, 마스크 착용 여부 판단 등에서 활용됨.

## 3. Landmark의 다양한 토폴로지
- **5-point landmark**
  - 두 눈, 코 끝, 입 양쪽.
  - 간단한 정렬(face alignment)에 자주 사용.
- **68-point landmark**
  - 얼굴 외곽, 눈썹, 눈, 코, 입 전체를 세밀하게 포함.
  - 표정 인식, 세밀한 얼굴 분석에 활용.
- **더 확장된 형태**
  - 98-point landmark (예: WFLW dataset)
  - 194-point landmark (고해상도 얼굴 연구용)
  - 특정 연구 목적(예: micro-expression 분석)에 맞추어 더 많은 점을 활용하기도 함.

<br>

68pts landmark 예시:

![68pts landmark 예시](https://www.researchgate.net/profile/Ana_Tanevska/publication/320979643/figure/fig3/AS:636979167383553@1528879076541/The-68-facial-landmarks-extracted-from-a-frontal-face-view.png)


<br>


## 4. 공개된 Face Dataset 예시
- **LFW (Labeled Faces in the Wild)**  
  - "in-the-wild" 환경에서 촬영된 얼굴 이미지 데이터셋. 얼굴 인식 벤치마크에 많이 쓰임.
- **CelebA**  
  - 20만 장 이상의 유명인 얼굴 이미지. 속성(attribute) 레이블 포함 (예: 안경 착용 여부, 미소 여부).
- **WFLW (Wider Facial Landmarks in-the-Wild)**  
  - 98점 landmark 제공. 다양한 표정, 조명, 가림 현상이 포함.
- **300W**  
  - 여러 landmark 데이터셋을 통합. 얼굴 landmark 검출 연구에서 표준으로 사용됨.
- **AffectNet**  
  - 감정 레이블이 포함된 대규모 얼굴 데이터셋. 감정 분석에 활용.

<br>

## 5. 이론과 코드의 연결
- 코드에서는 보통 **얼굴 검출(Face Detection)** 후 → **Landmark 추출** → 정렬 및 분석을 수행.
- Landmark는 단순히 점 좌표 집합이 아니라, 얼굴을 수학적으로 정렬하고 딥러닝 모델 입력을 정규화하는 데 필수.
- 따라서 코드 학습 전, landmark의 역할과 데이터셋 특성을 이해하는 것이 매우 중요함.


In [ ]:
!wget https://dimg.donga.com/wps/NEWS/IMAGE/2021/03/26/106095249.2.jpg

# OpenCV ViolaJones

In [ ]:
#https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html

In [ ]:
#https://github.com/kipr/opencv/tree/master/data/haarcascades
!wget https://github.com/kipr/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
face_cascade = cv2.CascadeClassifier()
face_cascade.load('haarcascade_frontalface_default.xml')

In [ ]:
bgr = cv2.imread('/content/106095249.2.jpg')
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

In [ ]:
gray

In [ ]:
gray = cv2.equalizeHist(gray)
#-- Detect faces
faces = face_cascade.detectMultiScale(gray)

In [ ]:
faces


In [ ]:
dst = bgr.copy()
for (x,y,w,h) in faces:
    cv2.rectangle(dst, (x, y), (x+w, y+h), (255, 0, 0), 3)
cv2_imshow(dst)

# 딥러닝 기반

In [ ]:
!pip install mtcnn

In [ ]:
import mtcnn

In [ ]:
detector = mtcnn.MTCNN()

In [ ]:
!wget https://img.hankyung.com/photo/202101/01.25062168.1.jpg -O girls.jpg

In [ ]:
image = cv2.imread('girls.jpg')
faces = detector.detect_faces(image[..., [2, 1, 0]])

In [ ]:
image

In [ ]:
faces[0]['box']
faces[0]['keypoints']


In [ ]:
dst = image.copy()
# for i in range(len(faces)):
#     face = faces[i]
for face in faces:
    x1, y1, w, h = face['box']
    x2, y2 = x1 + w, y1 + h
    cv2.rectangle(dst, (x1, y1), (x2, y2), (255, 0, 0), 2)

    for key, value in face['keypoints'].items():
        cv2.circle(dst, value, 2, (0, 0, 255), -1)
cv2_imshow(dst)



In [ ]:
from matplotlib import pyplot
from google.colab.patches import cv2_imshow
import cv2
image = cv2.imread('/content/106095249.2.jpg')
faces = detector.detect_faces(image[..., [2, 1, 0]])

dst = image.copy()
for face in faces:
    print(face)
    x1, y1, w, h = face['box']
    x2, y2 = x1 + w, y1 + h
    cv2.rectangle(dst, (x1, y1), (x2, y2), (0,0,255), 3)
    keypoints = face['keypoints']
    for part in ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']:
        point = keypoints[part]
        cv2.circle(dst, point, 3, (0, 255, 0), cv2.FILLED)

cv2_imshow(dst)

In [ ]:
import dlib
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
# https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=zzing0907&logNo=221612308385

In [ ]:
def print_circle(part, dst, color):
    cv2.circle(dst, (part.x, part.y), 3, color, -1)

image = cv2.imread('/content/106095249.2.jpg')
dst = image.copy()
rects = detector(image[..., [2, 1, 0]].copy(), 1)
for rect in rects:
    x1, x2, y1, y2 = rect.left(), rect.right(), rect.top(), rect.bottom()
    shape = predictor(image[..., [2, 1, 0]].copy(), rect)
    for j in range(17):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (0, 255, 0), -1)
    for j in range(17, 22):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (0, 0, 255), -1)
    for j in range(22, 27):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (255, 0, 0), -1)
    for j in range(27, 36):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (255, 255, 0), -1)
    for j in range(36, 42):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (0, 255, 255), -1)
    for j in range(42, 48):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (255, 0, 255), -1)
    for j in range(48, 68):
        part = shape.part(j)
        cv2.circle(dst, (part.x, part.y), 3, (255, 255, 255), -1)
    cv2.rectangle(dst, (x1, y1), (x2, y2), (0, 0, 255), 3)

    # index = [36, 45, 57]
    # for i in range(3):
    #     index1 = index[i]
    #     index2 = index[(i + 1) % 3]

    index = [(36, 45), (45, 57), (57, 36)]
    for index1, index2 in index:
        p1 = shape.part(index1)
        p2 = shape.part(index2)
        cv2.line(dst, (p1.x, p1.y), (p2.x, p2.y), (0, 0, 255), 3)


cv2_imshow(dst)

In [ ]:
# https://github.com/ageitgey/face_recognition
!pip install face_recognition

In [ ]:
import face_recognition

In [ ]:
def draw_points(dst, points, color):
    for point in points:
        cv2.circle(dst, point, 3, color, -1)

image = cv2.imread('/content/106095249.2.jpg')
# image = face_recognition.load_image_file("/content/106095249.2.jpg")
face_locations = face_recognition.face_locations(image[..., [2, 1, 0]])
face_landmarks_list = face_recognition.face_landmarks(image[..., [2, 1, 0]])
dst = image.copy()
for x1, y1, x2, y2 in face_locations:
    cv2.rectangle(dst, (x1, y1), (x2, y2), (0,0,255), 3)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (127, 127, 0), (0, 127, 127), (127, 0, 127)]
for landmarks in face_landmarks_list:
    for key, color in zip(landmarks.keys(), colors):
        draw_points(dst, landmarks[key], color)

    print(landmarks)

cv2_imshow(dst)

In [ ]:
face_landmarks_list

# 얼굴 처리 파이프라인

## 1) 얼굴 처리 파이프라인의 전체 흐름
- **Detection(검출)**: 이미지/영상에서 얼굴 위치를 찾음 → 바운딩 박스(x, y, w, h 또는 x1, y1, x2, y2).
- **Landmark(특징점 추정)**: 박스 내부에서 눈·코·입·윤곽 등 기준점 좌표(2D)를 추정.
- **Alignment(정렬)**: 랜드마크 기반으로 회전·기울기·스케일을 보정하여 표준 자세/크기로 맞춤.
- **Embedding/Recognition(임베딩·인식)**: 정렬된 얼굴을 벡터(예: 128D)로 변환 → 벡터 간 거리 비교로 동일인 여부 판정.

<br>

## 2) 검출(Detection) 이론 요약과 실전 유의점
### 2.1 전통 방식 (Haar Cascade / Viola–Jones)
- **구성**: Haar-like 특징 + AdaBoost 앙상블 + 캐스케이드.
- **장점**: 가볍고 빠름, 정면 표준 자세에 적합.
- **한계**: 포즈/조명/가림에 취약.

### 2.2 딥러닝 기반 (MTCNN, RetinaFace)
- **MTCNN**: P-Net → R-Net → O-Net, 박스 + 5점 랜드마크 제공.
- **RetinaFace**: 앵커 기반, 박스 + 풍부한 랜드마크 + 품질 정보 제공.

### 실무 팁
- OpenCV는 BGR, 딥러닝 프레임워크는 RGB → 변환 일관성 유지.
- 좌표계 혼용([x, y, w, h] vs [x1, y1, x2, y2]) → 내부 표준화 필요.
- 바운딩 박스 경계 넘어가면 클리핑 처리.

<br>


## 3) 랜드마크(Feature Points)
### 정의와 역할
- **랜드마크**: 눈꼬리, 콧등, 입꼬리, 턱선 등 해부학적 기준점 2D 좌표.
- **주요 용도**:
  - 정렬(Alignment) → 임베딩 품질 향상.
  - 표정/시선/자세 분석.
  - 합성·편집·AR 효과.

### 정렬(Alignment)의 수학적 핵심
- **유사변환(similarity transform)**: 회전 R + 스케일 s + 평행이동 t.
- **최적화 문제**:  
![formula](https://latex.codecogs.com/svg.latex?\min_{s,R,t}\|sRX+t-Y\|)  
  (프로크루스테스 정렬)
- **실무**: 5점(양눈·코·입 좌/우)으로 충분, 정렬 후 112×112 또는 224×224로 리사이즈.

<br>


## 4) 랜드마크 토폴로지(포인트 세트 설계)
- **3점**: 좌·우 눈 + 코끝.
- **5점**: 양눈 중심 + 코끝 + 입 좌/우 (가장 널리 쓰임).
- **21/27/33점**: 중간 밀도(데이터셋/SDK별).
- **68점 (dlib/iBUG 표준)**: 턱 17, 눈썹 10, 코 9, 눈 12, 입 20.
- **98점 (WFLW)**: 다양한 포즈/표정/가림 대응.
- **106점 (InsightFace)**: 98점 확장.
- **194/196점**: 눈꺼풀/입술 안쪽까지 포함.
- **468점 (MediaPipe Face Mesh)**: 메시 기반(2D지만 3D 정합 가능).
- **Dense Mesh**: 수천~수만 점, 3D Morphable Model 등.

### 선택 기준
- **정렬/인식 중심** → 5점.
- **표정/윤곽 분석** → 68/98/106점.
- **AR/메이크업/정밀 편집** → 196/468점.

<br>


## 5) 얼굴 임베딩/인식
- CNN(ResNet, MobileFaceNet 등) → 고정 길이 벡터(128D, 256D).
- 학습: 대규모 아이덴티티 분류 + Metric Loss (ArcFace, CosFace).
- 추론: 두 벡터 간 거리(코사인/유클리드) 비교 → 동일인 여부 판정.

<br>


## 6) 데이터셋 실무 처리
- **채널**: RGB 기준, OpenCV는 BGR→RGB 변환 필요.
- **좌표계**: 내부 표준화 및 변환 유틸 고정.
- **정렬 크기**: 112×112, 224×224 등 통일.
- **라벨 일관성**: 동일 기준 유지, 어노테이션 검수 필수.
- **스플릿 규칙**: 동일 인물은 train/val/test에 동시에 포함 금지.

<br>


## 7) 평가 지표
- **검출**: Precision/Recall, mAP.
- **랜드마크**: NME, AUC@0.08, Failure Rate.
- **인식**: ROC, EER, TPR@FPR.

<br>


## 8) 강의용 샘플 이미지
- **정면-밝음**: 표준 품질 (정렬 전/후 비교).
- **정면-어두움/역광**: 조명 영향 시연.
- **측면 ±30°/±60°**: 랜드마크 밀도 차이 비교.
- **가림 (마스크/손/안경)**: 실패 사례.
- **군중/원거리 다인 이미지**: NMS/중복 검출 사례.


<br>


## 9) 공개 Face 데이터셋
### 검출(Detection)
- **WIDER FACE**: 다양한 포즈/규모/가림.
- **FDDB**: 전통적 벤치마크.
- **MAFA**: 마스크 착용 얼굴.

### 랜드마크(Landmark)
- **300-W**: 68점 표준.
- **WFLW**: 98점 표준, 속성별 평가.
- **AFLW**: 광범위 포즈·표정.
- **COFW**: 가림 상황 특화.
- **Menpo 2D/3D, LaPa**: 2D/3D 및 세밀 랜드마크 포함.

### 인식(Recognition)
- **LFW**: 전통적 벤치마크.
- **VGGFace2**: 대규모, 다양한 포즈·표정.
- **CASIA-WebFace**: 학습용 레퍼런스.
- **MS1M (MS1MV2)**: 대규모 웹 얼굴 데이터.
- **IJB-A/B/C**: 고난도 검증/식별 벤치마크.

<br>


## 10) 실무 디자인 패턴
- **Annotation Assistant**: 박스 기반 자동 랜드마크 추정.
- **Detector + Landmark 하이브리드**: YOLO/RetinaFace → 랜드마크 → 인식.
- **정렬 후 표준화**: 캐노니컬 좌표 기준 112×112/224×224 고정.
- **평가 프로토콜 분리**: Detection/AP, Landmark/NME, Recognition/EER.

<br>

## 11) SAM 및 생태계 도구 접점
- **SAM (Segment Anything)**: 얼굴 윤곽 마스크 자동 추출, 편집/라벨링 보조.
- **Hugging Face**: 모델 가중치 및 데모 공유.
- **Ultralytics**: YOLO 기반 파이프라인 통합.

연계 패턴:  
**검출(박스) → 랜드마크 정렬 → 인식/분류 → 필요 시 SAM으로 윤곽 마스크 추가**
